<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/instagramm_pars_07022019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import hashlib
import json
import re
import requests


count=36
session = requests.Session()

      
def update(obj=None, pointer=None, idx=None, rhx_gis=None, settings={}):

      src_adress = [] 
      base_url = ""
      query = "https://www.instagram.com/"
      if not obj is None:
            query += base_url + obj
            
        
      if pointer is None: 
        response = session.get(query, **settings)
        match = re.search(r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>", response.text)
        data = json.loads(match.group(1))
        rhx_gis = data["rhx_gis"]
        csrf_token = data["config"]["csrf_token"]
        idx = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["id"]
        
        #print("end_cursor", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])
        #print("'has_next_page", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'])
        
        for number in range(len(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"])):
	          src_adress.append(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["thumbnail_resources"][4]["src"])
        if data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:                 
            pointer = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        else:
            pointer = None

        return rhx_gis, pointer, src_adress, idx, csrf_token

      
      else:
        variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        while True:
            data = {"id": idx, "first": 36, "after": pointer}
            variables=variables_string.format(**data)
            query_hash="c6809c9c025875ac6f02619eae97a80e"
            #response = graphql_request(query_hash="c6809c9c025875ac6f02619eae97a80e", variables=variables, settings=settings)
            settings["params"] = {"query_hash": query_hash}
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            response = session.get("https://www.instagram.com/graphql/query/", **settings)
            response.raise_for_status()
            
            data = response.json()["data"]["user"]["edge_owner_to_timeline_media"]

            edges = data["edges"]
            page_info = data["page_info"]
            length = len(edges)

            for index in range(length):
                    node = edges[index]["node"]
                    shortcode = node["shortcode"]        #["src"]
                    if not "shortcode" in node:
                        length -= 1
                        continue
                    m = node["display_url"]
                    #m._set_data(node)
                    src_adress.append(m)
                    #print("node", node)
                
            if page_info["has_next_page"]:
                    
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None

            return src_adress, pointer

      

In [0]:
src_adress = []
rhx_gis, pointer, src, idx, csrf_token = update("likekate_")
src_adress.extend(src)
print("rhx_gis:", rhx_gis, "csrf_token:", csrf_token, "idx =", idx)
for i in range(100):
  src, pointer = update("likekate_", pointer=pointer, idx=idx, rhx_gis=rhx_gis)
  src_adress.extend(src)
  print(len(src_adress))


In [4]:
import hashlib
import json
import re
import requests
session = requests.Session()
# rhx_gis: fc2e73d4fd7dddcd31d28bea5cb2df59 csrf_token: 23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv
def AgentAccount(login=None, password=None, settings={}):
        csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
        login = "andivanov2019"
        password = "Asus2046"
        data = {"username": login, "password": password}
        
        if "headers" in settings:
            #settings["params"] = {"query_hash": query_hash}
            settings["headers"]["X-CSRFToken"] = csrf_token
            settings["headers"]["referer"] = "https://www.instagram.com/"
        else:
            settings["headers"] = {
                "X-CSRFToken": csrf_token,
                "referer": "https://www.instagram.com/",
            }

        response = session.post("https://www.instagram.com/accounts/login/ajax/", data, **settings)
        #response.raise_for_status()
        print("response:", response)
        try:
            data = response.json()
            if not data["authenticated"]:
                raise AuthException(login) 
        except (ValueError, KeyError) as exception:
            print("ERROR1")

AgentAccount()

response: <Response [200]>


In [5]:
#query_hash:56066f031e6239f35a904ac20c9f37d9
#variables:{"id":"10180157718","include_reel":true,"fetch_mutual":false,"first":12,"after":""}




def get_followers(account=None, pointer=None, count=1000, settings={}, limit=50):
  
        #update(account, settings)
        csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
        rhx_gis = "fc2e73d4fd7dddcd31d28bea5cb2df59"
        #query_hash = "QVFENklrNmY2Y2JvRGhEa2k5UV9XZTVkRHcwSDB6OXM3dWhhZkVsSXpNdlZacEJQVU9QVkcybDQtaC11c04tSXN3cWlmUzVIN3NxeG1BR2phV0VYSU1fcw=="
        query_hash="37479f2b8209594dde7facb0d904896a"
        idx = "2120762022"

        if pointer is None:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first}}}'
            variables_string = '{{"id":"{id}","first":{first}}}'
        else:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first},"after":"{after}"}}'
            variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        followers = []

        while True:
            data = {"first": min(limit, count), "id": idx}
            if not pointer is None:
                data["after"] = pointer
            settings["params"] = {"query_hash": query_hash}
            variables=variables_string.format(**data)
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            
            i = 2
            for i > 0:
              try:
                response = session.get("https://www.instagram.com/graphql/query/", **settings)
                response.raise_for_status()
              except (ValueError, KeyError) as exception:
                raise print("ERROR")
                if exception.response.status_code == 200: 
                   continue
                else:
                   exception.response.status_code == 429:
                    print("sleep") 
                    sleep(60) 
                    print("end sleep")
            try:
            data = response.json()
            #print(data)
            data = response.json()["data"]["user"]["edge_followed_by"]
            edges = data["edges"]
            page_info = data["page_info"]
            #print(page_info)
            followers_count = data["count"]
                
            for index in range(min(len(edges), count)):
                    node = edges[index]["node"]
                    a = node["username"]
                    id = node["id"]
                    profile_pic_url = node["profile_pic_url"]
                    is_verified = node["is_verified"]
                    full_name = node["full_name"]
                    followers.append(a)
                    if len(followers) % 500 == 0:
                      print(len(followers))
                
            if page_info["has_next_page"]:
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None
                
            if len(edges) < count and page_info["has_next_page"]:
                    count = count-len(edges)
                    variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
            else:
                    return followers, pointer
            
                
get_followers("likekate_")

{'data': {'user': {'edge_followed_by': {'count': 309918, 'page_info': {'has_next_page': True, 'end_cursor': 'QVFDVS1zRURQaUJteWxZblN5RDAxckVIajdsVy1pOGZoZG9GTktsamFhbVE4aGZfQ0xrNjliZDdCMWo5LVg4R1QwX3NpdklOWGdPejd2cHVEdmtSbllIVA=='}, 'edges': [{'node': {'id': '8287211612', 'username': 'ywsyslwm58', 'full_name': 'יוסי שלום', 'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/39f884ec13305746cbb4bbb0e03bfea8/5CE3DCCC/t51.2885-19/s150x150/47694585_212571872902040_2690425187035250688_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'is_verified': False, 'followed_by_viewer': False, 'requested_by_viewer': False}}, {'node': {'id': '4289349057', 'username': 'ct.lika', 'full_name': 'Ｌｉｋａ', 'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/1e7179f1ab10280085acd93e8cae0572/5CE4561F/t51.2885-19/s150x150/28766035_561024730962128_4647031907056353280_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'is_verified': False, 'followed_by_viewer': False, 'requested_by_viewer': Fals

(['ywsyslwm58',
  'ct.lika',
  'alekseybezgubyy',
  'czirjaktomi',
  'igorumarkhadzhie',
  'nor_bort',
  'yoyo1478',
  'cv.mber',
  '__valeriya_official_',
  'fit.miss_n',
  '_huseynov_bextiyar_22',
  'aftodieva___',
  'nauchu_zarobotku',
  'amiran_purtseladze',
  'alinkachetverikova',
  'wioletaderda',
  'ibrahim_gurses',
  'ahadov218',
  'james_cashmoney_official',
  'sirena222',
  'kevinpacey7',
  'cipilxc',
  'valentin.mihai0',
  'omerarslanorj',
  'jessicajnn',
  'enabbella',
  'donataszubenas',
  'kozaczek1337_',
  'karen.8931',
  'lieneliene333',
  'alison.cln',
  'desiktirlo2121',
  'amylovessammy',
  'aurora_38_irk',
  'bitcoin_best_invest',
  'shima_stuttgart',
  'bozhenko4059',
  'salon_newton',
  'limalinami',
  'alexbiker',
  'franta_gabriela',
  'agniatorekhan',
  'sarammccabe',
  'sirena_by',
  'efemakul',
  'nates__insta',
  'a_f_r_i_z_o_n',
  'body_masters_fitness',
  'tattoo.777',
  'a.minni.a',
  'alimuradova__________',
  'lovetravelllll',
  'solovajka',
  'nogti_st

In [0]:
followers

NameError: ignored

In [0]:
get_followers("likekate_", pointer="QVFDSGVsXzh6MjVkWW5Oc1dNQ1RVcFlkOU5WTDFBcHVoMGozQXpGVjJvU1Q3M1RPc3lhMDRwaS1SYm9uVjdFY2dkZmU1eWtScUZ6cXc3YURMT2k3RnZkZw==")

{'has_next_page': True, 'end_cursor': 'QVFDUWk3TF9OUUxyaENLWklnS3JTR21XcjNxSzBIeDVwRzFFbzBPR2U3R1gzMkNYRnZEX0stX2JnVy11a1FEY2t1Mzloc05RbXp1dGk4eG5PQWRUTC1HcQ=='}


(['otravochka',
  'aleshakamov',
  'morris.rnb',
  'tamarabentalder',
  '_loveinna',
  'mbhh85567',
  'aleksandr_official_moskva',
  'ildarovrrr',
  'al_sdlv',
  'kolibrik_tattoo',
  'milanka443',
  'myata.kazakova',
  'alina_timoshkova',
  'ashchetsova',
  'gabriela_ferraz19',
  'myatalounge_fili',
  'miu_miu_074',
  'shuhrat6743',
  'jeanchristophemarie6',
  'ulyanaburyak'],
 'QVFDUWk3TF9OUUxyaENLWklnS3JTR21XcjNxSzBIeDVwRzFFbzBPR2U3R1gzMkNYRnZEX0stX2JnVy11a1FEY2t1Mzloc05RbXp1dGk4eG5PQWRUTC1HcQ==')